In [31]:
using System;
using System.Collections.Generic;

public class Intervalo
{
	public string flag {get; set;}
	public Intervalo(string flag)
	{
		this.flag = flag;
	}
}

public class Bucket
{
	public string flagInicial { get; set; } = "";
	public string flagFinal { get; set; } = "";
	public List<Intervalo> intervalos {get; set;}
	
	public Bucket()
	{
		intervalos = new List<Intervalo>();
	}

	public Bucket(string flagInicial, string flagFinal, List<Intervalo> intervalos)
	{
		this.flagInicial = flagInicial;
		this.flagFinal = flagFinal;
		this.intervalos = intervalos;
	}
}

public void printList(List<Intervalo> list)
{
	Console.Write("[ ");
	list.ForEach((listItem) => {
		if(list.IndexOf(listItem) == list.Count - 1)
		{
			Console.Write(listItem.flag.ToString());
			return;
		}
		Console.Write(listItem.flag.ToString() + " , ");			
	});
	Console.Write(" ]");
}

public void printBucket(Bucket bucket)
{
	Console.Write("[");
	Console.Write(bucket.flagInicial);
	Console.Write("]");
	Console.Write("[");
	Console.Write(bucket.flagFinal);
	Console.Write("]");
	printList(bucket.intervalos);
}

public void calcularBuckets(List<Bucket> bucketList, List<Intervalo> intervaloList){
	// Criar balde para armazenar cada período
	Bucket bucket = new Bucket();
	Intervalo fimAnterior = null;
	intervaloList.ForEach((listItem) => {
		// Se for um início ou período e teve um fim antes
		if((listItem.flag == "P" || listItem.flag == "I") && fimAnterior != null && !string.IsNullOrEmpty(bucket.flagFinal))
		{
			bucketList.Add(bucket);
			bucket = new Bucket();
			fimAnterior = null;
		}
		
		// Se for um período
		if(listItem.flag == "P")
		{
			bucket.intervalos.Add(listItem);
		}

		// Se for um início
		if(listItem.flag == "I")
		{
			// Se teve um início logo antes
			if(!string.IsNullOrEmpty(bucket.flagInicial))
			{
				bucket.intervalos.Add(listItem);
			}
			bucket.flagInicial = listItem.flag;
		}

		// Se for um fim
		if(listItem.flag == "F")
		{
			if(!string.IsNullOrEmpty(bucket.flagFinal))
			{
				bucket.intervalos.Add(fimAnterior);	
			}

			fimAnterior = listItem;
			bucket.flagFinal = listItem.flag;
		}

		// Caso seja o último e não tenha mais um final no dia
		if(intervaloList.IndexOf(listItem) == intervaloList.Count - 1)
		{
			bucketList.Add(bucket);
			fimAnterior = null;
			bucket = null;
			
			// Tratar dia só com períodos
			if(bucketList.Count == 1){	
				// Tratar dia sem início ou fim				
				if(string.IsNullOrEmpty(bucketList[0].flagInicial) && string.IsNullOrEmpty(bucketList[0].flagFinal) && bucketList[0].intervalos.Count > 0){
					bucketList[0].flagInicial = "ID"; // Início do dia
					bucketList[0].flagFinal = "FD"; // Fim do dia
				}

				// Tratar dia só com início
				if(!string.IsNullOrEmpty(bucketList[0].flagInicial) && string.IsNullOrEmpty(bucketList[0].flagFinal)){
					bucketList[0].flagFinal = "FD"; // Fim do dia
				}
				
				// Tratar dia só com fim
				if(string.IsNullOrEmpty(bucketList[0].flagInicial) && !string.IsNullOrEmpty(bucketList[0].flagFinal)){
					bucketList[0].flagInicial = "ID"; // Início do dia
				}				
			}			
		}
	});
	Console.WriteLine("Intervalos");
	printList(intervaloList);
	Console.WriteLine();
	Console.WriteLine("Buckets");
	bucketList.ForEach((listItem) => {
		printBucket(listItem);
		Console.WriteLine();
	});
}

public List<Intervalo> criarListaIntervalos(List<string> list){
	List<Intervalo> intervaloList = new List<Intervalo>();

	list.ForEach((listItem) => {
		intervaloList.Add(new Intervalo(listItem));
	});
	return intervaloList;
}

// Tentar gerar algoritmo para dividir intervalos de tempo
// marcados com flags em baldes/caixas para auxiliar nos
// cálculos de condução
List<Intervalo> intervaloList = new List<Intervalo>();
List<Bucket> bucketList = new List<Bucket>();

// Situação com linha do tempo no estado: [ P, P, F, F, I, I, P, F, I, P, P ]
// Baldes esperados: [ P, P, F, F ], [ I, I, P, F ], [ I, P, P ]
// Baldes esperados com nova notação: [][F][ P, P, F ], [I][F][ I, P ], [I][][ P, P ]
// Situação em que o período começa antes do dia e termina depois do dia
intervaloList = criarListaIntervalos(new List<string>{"P", "P", "F", "F", "I", "I", "P", "F", "I", "P", "P"});

Console.WriteLine("Situação começando antes do dia e terminando no dia: ");
calcularBuckets(bucketList, intervaloList);
bucketList.Clear();


// Situação em que todos os intervalos começam e terminam no dia (Ideal)
// Situação com linha do tempo no estado: [ I, P, P, F, I, P, F, I, P, P, F ]
// Baldes esperados: [ I, P, P, F ], [ I, P, F ], [ I, P, P, F ]
// Baldes esperados com nova notação: [I][F][ P, P ], [I][F][ P ], [I][F][ P, P ]
intervaloList = criarListaIntervalos(new List<string>{"I", "P", "P", "F", "I", "P", "F", "I", "P", "P", "F"});

Console.WriteLine("Situação começando e terminando no dia: ");
calcularBuckets(bucketList, intervaloList);
bucketList.Clear();

// Situação em que um intervalo começa antes e todos os outros terminam no dia
// Situação com linha do tempo no estado: [ P, P, F, I, P, F, I, P, P, F ]
// Baldes esperados: [ P, P, F ], [ I, P, F ], [ I, P, P, F ]
// Baldes esperados com nova notação: [][F][ P, P ], [I][F][ P ], [I][F][ P, P ]
intervaloList = criarListaIntervalos(new List<string>{"P", "P", "F", "I", "P", "F", "I", "P", "P", "F"});

Console.WriteLine("Situação começando antes do dia e terminando no dia: ");
calcularBuckets(bucketList, intervaloList);
bucketList.Clear();

// Situação em que um intervalo termina depois e todos os outros começam e terminam no dia
// Situação com linha do tempo no estado: [ I, P, P, F, I, P, F, I, P, P ]
// Baldes esperados: [ I, P, P, F ], [ I, P, F ], [ I, P, P ]
// Baldes esperados com nova notação: [I][F][ P, P ], [I][F][ P ], [I][][ P, P ]
intervaloList = criarListaIntervalos(new List<string>{"I", "P", "P", "F", "I", "P", "F", "I", "P", "P"});

Console.WriteLine("Situação começando no dia e terminando depois do dia: ");
calcularBuckets(bucketList, intervaloList);
bucketList.Clear();

// Situação em que um intervalo termina depois e todos os outros começam e terminam no dia
// Situação com linha do tempo no estado: [ I, P, P, F, P, P, I, P, F, I, P, P, F ]
// Baldes esperados: [ I, P, P, P, P, F ], [ I, P, F ], [ I, P, P, F ]
// Baldes esperados com nova notação: [I][F][ P, P, P, P ], [I][F][ P ], [I][F][ P, P ]
intervaloList = criarListaIntervalos(new List<string>{"I", "P", "P", "F", "P", "P", "I", "P", "F", "I", "P", "P", "F"});

Console.WriteLine("Situação com períodos entre fim e início de entre jornada: ");
calcularBuckets(bucketList, intervaloList);
bucketList.Clear();

// Situação em que todos os intervalos são períodos
// Situação com linha do tempo no estado: [ P, P, P, P, P, P, P ]
// Baldes esperados: [ P, P, P, P, P, P, P ]
// Baldes esperados com nova notação: [][][ P, P, P, P, P, P, P ]
intervaloList = criarListaIntervalos(new List<string>{ "P", "P", "P", "P", "P", "P", "P" });

Console.WriteLine("Situação em que todos os intervalos são períodos ");
calcularBuckets(bucketList, intervaloList);
bucketList.Clear();

Situação começando antes do dia e terminando no dia: 
Intervalos
[ P , P , F , F , I , I , P , F , I , P , P ]
Buckets
[][F][ P , P , F ]
[I][F][ I , P ]
[I][][ P , P ]
Situação começando e terminando no dia: 
Intervalos
[ I , P , P , F , I , P , F , I , P , P , F ]
Buckets
[I][F][ P , P ]
[I][F][ P ]
[I][F][ P , P ]
Situação começando antes do dia e terminando no dia: 
Intervalos
[ P , P , F , I , P , F , I , P , P , F ]
Buckets
[][F][ P , P ]
[I][F][ P ]
[I][F][ P , P ]
Situação começando no dia e terminando depois do dia: 
Intervalos
[ I , P , P , F , I , P , F , I , P , P ]
Buckets
[I][F][ P , P ]
[I][F][ P ]
[I][][ P , P ]
Situação com períodos entre fim e início de entre jornada: 
Intervalos
[ I , P , P , F , P , P , I , P , F , I , P , P , F ]
Buckets
[I][F][ P , P ]
[I][F][ P , P , P ]
[I][F][ P , P ]
Situação em que todos os intervalos são períodos 
Intervalos
[ P , P , P , P , P , P , P ]
Buckets
[ID][FD][ P , P , P , P , P , P , P ]
